## Installation of TwelveLabs SDKs

In [ ]:
!pip install -U -q twelvelabs

## Configure your API key
Add API Key as a env variable. Signup for TwelveLabs and Get your API keys [here](https://playground.twelvelabs.io/?_gl=1*1rx2bxa*_ga*MTUzMTIzMTI3MC4xNzI3OTAxMzE2*_ga_END0TB2RFD*MTc0MDE4NTM2NS4yNC4xLjE3NDAxODY1MjkuMC4wLjA.). No credit card is required to use the Free plan. The Free plan includes indexing of 600 mins of videos, which is enough for a small project.

### Create an AWS Secrets Manager secret 

Follow steps here to [create a secret](https://docs.aws.amazon.com/secretsmanager/latest/userguide/create_secret.html) in AWS Secrets Manager. For example, name the secret as `TL_API_Key`.

Note down the ARN or name of the secret (`TL_API_Key`) to retrieve. To retrieve a secret from another account, you must use an ARN.

For an ARN, we recommend that you specify a complete ARN rather than a partial ARN. See [Finding a secret from a partial ARN](https://docs.aws.amazon.com/secretsmanager/latest/userguide/troubleshoot.html#ARN_secretnamehyphen).

Use the name of the secret for the `SecretId` in the code block below.

In [ ]:
import boto3
import json
secrets_manager_client=boto3.client("secretsmanager")
API_secret=secrets_manager_client.get_secret_value(
    SecretId="TL_API_KEY"
)
TL_API_KEY=json.loads(API_secret["SecretString"])["TL_API_Key"]

## Using video data

For the demo, you will on these videos: [Robin bird forest Video by Federico Maderno from Pixabay](https://pixabay.com/videos/robin-bird-forest-nature-spring-21723/) and [Island Video by Bellergy RC from Pixabay](https://pixabay.com/videos/island-motorboat-sea-ocean-2946/). 

This demo depends upon some video data. To use this, we will download 2 mp4 files and upload it to an Amazon S3 bucket.

1. Click on the links containing the [Robin bird forest Video by Federico Maderno from Pixabay](https://pixabay.com/videos/robin-bird-forest-nature-spring-21723/) and [Island Video by Bellergy RC from Pixabay](https://pixabay.com/videos/island-motorboat-sea-ocean-2946/) videos.
1. Download the 21723-320725678_small.mp4 and 2946-164933125_small.mp4 files.
1. Create an S3 bucket if you don't have one already. Follow the steps in the [Creating a bucket doc](https://docs.aws.amazon.com/AmazonS3/latest/userguide/create-bucket-overview.html). Note down the name of the bucket and replace it the code block below (Eg., `MYS3BUCKET`).
1. Upload the 21723-320725678_small.mp4 and 2946-164933125_small.mp4 video files to the S3 bucket created in the step above by following the stesp in the [Uploading objects doc](https://docs.aws.amazon.com/AmazonS3/latest/userguide/upload-objects.html). Note down the name of the objects and replace it the code block below (Eg., `21723-320725678_small.mp4` and `2946-164933125_small.mp4`)


In [ ]:
s3_client=boto3.client("s3")
bird_video_data=s3_client.download_file(Bucket='MYS3BUCKET',  Key='21723-320725678_small.mp4', Filename='robin-bird.mp4')
island_video_data=s3_client.download_file(Bucket='MYS3BUCKET',  Key='2946-164933125_small.mp4', Filename='island.mp4')

## Generate Embeddings
Use the Embed API to create multimodal embeddings that are contextual vector representations for your videos and texts. Twelve Labs video embeddings capture all the subtle cues and interactions between different modalities, including the visual expressions, body language, spoken words, and the overall context of the video, encapsulating the essence of all these modalities and their interrelations over time.

To create video embeddings, you must first upload your videos, and the platform must finish processing them. Uploading and processing videos require some time. Consequently, creating embeddings is an asynchronous process comprised of three steps:

1. Upload and process a video: When you start uploading a video, the platform creates a video embedding task and returns its unique task identifier.

2. Monitor the status of your video embedding task: Use the unique identifier of your task to check its status periodically until it's completed.

3. Retrieve the embeddings: After the video embedding task is completed, retrieve the video embeddings by providing the task identifier.
Learn more in the [docs](https://docs.twelvelabs.io/docs/create-video-embeddings)

In [ ]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import EmbeddingsTask, SegmentEmbedding

def print_segments(segments: List[SegmentEmbedding], max_elements: int = 1024):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

# Initialize client with API key
twelvelabs_client = TwelveLabs(api_key=TL_API_KEY)

video_files=["robin-bird.mp4", "island.mp4"]
tasks=[]

for video in video_files:
    # Create embedding task
    task = twelvelabs_client.embed.task.create(
        model_name="Marengo-retrieval-2.7",
        video_file=video
    )
    print(
        f"Created task: id={task.id} engine_name={task.model_name} status={task.status}"
    )
    
    def on_task_update(task: EmbeddingsTask):
        print(f"  Status={task.status}")
    
    status = task.wait_for_done(
        sleep_interval=2,
        callback=on_task_update
    )
    print(f"Embedding done: {status}")
    
    # Retrieve and inspect results
    task = task.retrieve()
    if task.video_embedding is not None and task.video_embedding.segments is not None:
        print_segments(task.video_embedding.segments)
    tasks.append(task)


## Create OpenSearch Domain

If you have an OpenSearch Service domain already, you can use the connection details for that domain. If you want to create a new domain, follow the steps in the [docs](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/createupdatedomains.html).

Follow [Operational best practices for Amazon OpenSearch Service](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/bp.html)

In [ ]:
!pip install opensearch-py
!pip install botocore
!pip install requests-aws4auth

## Getting OpenSearch Domain URL
Update your domain endpoint in the `host` section below.

You can find the domain endpoint under 'General Information' within details of domain created earlier.

Example: https://search-mydomain-1a2a3a4a5a6a7a8a9a0a9a8a7a.us-east-1.es.amazonaws.com

[Create an AWS Secrets Manager secret](https://docs.aws.amazon.com/secretsmanager/latest/userguide/create_secret.html) for the Amazon OpenSearch username and password.

Note down the ARN or name of the secret (Eg., `AOS_password`, `AOS_username`) to retrieve . To retrieve a secret from another account, you must use an ARN.

For an ARN, we recommend that you specify a complete ARN rather than a partial ARN. See [Finding a secret from a partial ARN](https://docs.aws.amazon.com/secretsmanager/latest/userguide/troubleshoot.html#ARN_secretnamehyphen).

In [ ]:
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
from requests.auth import HTTPBasicAuth

# OpenSearch connection configuration
# host = 'your-host.aos.us-east-1.on.aws'
host = 'your-host.aos.us-east-1.on.aws'
port = 443  # Default HTTPS port

# Get OpenSearch credentials from Secrets Manager
secrets_manager_client = boto3.client('secretsmanager')

opensearch_username = secrets_manager_client.get_secret_value(
    SecretId="AOS_username"
)
opensearch_username_string = json.loads(opensearch_username["SecretString"])["AOS_username"]

opensearch_password = secrets_manager_client.get_secret_value(
    SecretId="AOS_password"
)
opensearch_password_string = json.loads(opensearch_password["SecretString"])["AOS_password"]

auth = (opensearch_username_string, opensearch_password_string)

# Create the client configuration
client_aos = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# Test the connection
try:
    cluster_info = client_aos.info()
    print("Successfully connected to OpenSearch")
    print(f"Cluster info: {cluster_info}")
except Exception as e:
    print(f"Connection failed: {str(e)}")

# Define the enhanced index configuration
index_name = 'twelvelabs_index'
new_vector_index_definition = {
    "settings": {
        "index": {
            "knn": "true",
            "number_of_shards": 1,
            "number_of_replicas": 0
        }
    },
    "mappings": {
        "properties": {
            "embedding_field": {
                "type": "knn_vector",
                "dimension": 1024
            },
            "video_title": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword"
                    }
                }
            },
            "segment_start": {
                "type": "date"
            },
            "segment_end": {
                "type": "date"
            },
            "segment_id": {
                "type": "text"
            }
        }
    }
}

# Create index if it doesn't exist
client_aos.indices.create(index=index_name, body=new_vector_index_definition, ignore=400)

# Function to perform vector search
def search_similar_segments(query_vector, k=5):
    query = {
        "size": k,
        "_source": ["video_title", "segment_start", "segment_end", "segment_id"],
        "query": {
            "knn": {
                "embedding_field": {
                    "vector": query_vector,
                    "k": k
                }
            }
        }
    }
    
    response = client_aos.search(
        index=index_name,
        body=query
    )

    results = []
    for hit in response['hits']['hits']:
        result = {
            'score': hit['_score'],
            'title': hit['_source']['video_title'],
            'start_time': hit['_source']['segment_start'],
            'end_time': hit['_source']['segment_end'],
            'segment_id': hit['_source']['segment_id']
        }
        results.append(result)

    return (results)

# Function to format search results
def print_search_results(results):
    print("\nSearch Results:")
    print("-" * 50)
    for i, result in enumerate(results, 1):
        print(f"\nResult {i}:")
        print(f"Video: {result['title']}")
        print(f"Time Range: {result['start_time']} - {result['end_time']}")
        print(f"Similarity Score: {result['score']:.4f}")

# Example usage
if __name__ == "__main__":
    # Check if index exists
    if client_aos.indices.exists(index=index_name):
        print(f"Index '{index_name}' configuration:")
        print(json.dumps(client_aos.indices.get(index=index_name), indent=2))
    else:
        print(f"Index '{index_name}' does not exist")


## Upsert the video embeddings into OpenSearch

In [ ]:
from opensearchpy.helpers import bulk

def generate_actions(tasks, video_files):
    count = 0
    for task in tasks:
        # Check if video embeddings are available
        if task.video_embedding is not None and task.video_embedding.segments is not None:
            embeddings_doc = task.video_embedding.segments
            
            # Generate actions for bulk indexing
            for doc_id, elt in enumerate(embeddings_doc):
                yield {
                    '_index': index_name,
                    '_id': doc_id,
                    '_source': {
                        'embedding_field': elt.embeddings_float,
                        'video_title': video_files[count],
                        'segment_start': elt.start_offset_sec,
                        'segment_end': elt.end_offset_sec,
                        'segment_id': doc_id
                    }
                }
        print(f"Prepared bulk indexing data for task {count}")
        count += 1

# Perform bulk indexing
try:
    success, failed = bulk(client_aos, generate_actions(tasks, video_files))
    print(f"Successfully indexed {success} documents")
    if failed:
        print(f"Failed to index {len(failed)} documents")
except Exception as e:
    print(f"Error during bulk indexing: {e}")

## Create Text Embeddings and Perform Text-to-Video Search

In [ ]:
def print_segments(segments: List[SegmentEmbedding], max_elements: int = 1024):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

# Create text embeddings
text_res = twelvelabs_client.embed.create(
  model_name="Marengo-retrieval-2.7",
  text="Bird eating food",
)

print("Created a text embedding")
print(f" Model: {text_res.model_name}")

if text_res.text_embedding is not None and text_res.text_embedding.segments is not None:
        print_segments(text_res.text_embedding.segments)

In [ ]:
vector_search = text_res.text_embedding.segments[0].embeddings_float
print (vector_search)

In [ ]:
# Perform vector search
query_vector = vector_search
text_to_video_search = search_similar_segments(query_vector)
# print(text_video_search)
    
print_search_results(text_to_video_search)

## Create Audio Embeddings and Perform Audio-to-Video Search

You will work on this audio data to showcase audio to video search: [Rock 808 beat.mp3 by T_roy_920 -- License: Creative Commons 0](https://freesound.org/people/T_roy_920/sounds/425556/). We have downloaded the audio locally and uploaded it to the S3 bucket created earlier.

In [ ]:
audio_data=s3_client.download_file(Bucket='krao-test-one',  Key='425556__t_roy_920__rock-808-beat.mp3', Filename='audio-data.mp3')

In [ ]:
# Create audio embeddings
audio_res = twelvelabs_client.embed.create(
    model_name="Marengo-retrieval-2.7",
    audio_file="audio-data.mp3",
)

print(f"Created audio embedding: model_name={audio_res.model_name}")
print(f" Model: {audio_res.model_name}")

if audio_res.audio_embedding is not None and audio_res.audio_embedding.segments is not None:
    print_segments(audio_res.audio_embedding.segments)

In [ ]:
vector_search = audio_res.audio_embedding.segments[0].embeddings_float
print (vector_search)

In [ ]:
# Perform vector search
query_vector = vector_search
audio_to_video_search = search_similar_segments(query_vector)
# print(text_video_search)
    
print_search_results(audio_to_video_search)

## Create Image Embeddings and Perform Image-to-Video Search

You will work on this image data to showcase image to video search: [Ocean Image by shotbyjagar from Pixabay](https://pixabay.com/photos/ocean-water-sea-waves-seascape-5294270/). Download the image locally and uploaded it to the S3 bucket created earlier.

In [ ]:
# Create image embeddings
image_data=s3_client.download_file(Bucket='krao-test-one',  Key='ocean-5294270_1280.jpg', Filename='image-data.jpg')

image_res = twelvelabs_client.embed.create(
    model_name="Marengo-retrieval-2.7",
    image_file="image-data.jpg",
)

print(f"Created image embedding: model_name={image_res.model_name}")
print(f" Model: {image_res.model_name}")

if image_res.image_embedding is not None and image_res.image_embedding.segments is not None:
    print_segments(image_res.image_embedding.segments)

In [ ]:
vector_search = image_res.image_embedding.segments[0].embeddings_float
print (vector_search)

In [ ]:
# Perform vector search
query_vector = vector_search
image_to_video_search = search_similar_segments(query_vector)
# print(text_video_search)
    
print_search_results(image_to_video_search)